In [619]:
from curses import flash
import os
from pickle import TRUE
from re import L
import requests
import validators
import gzip
import shutil
from urllib.parse import urlparse
import random

import sys
import math
import xarray as xa
import numpy as np
from rio_cogeo import cog_validate
import rioxarray

# Mapping
import matplotlib as mpl
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

In [648]:
def open_file(input_path):
    return xa.open_dataset(input_file_path, engine="netcdf4", decode_coords='all', decode_times=False)

def copy_lat_lon_data(file, var_name1, var_name2, var_name3):
    lat = file[var_name1].data.copy()
    lat = lat[::-1]
    lon = file[var_name2].data.copy()
    data = file[var_name3].data[0].copy()
    return lat, lon, data

def make_new_xarray(lat, lon, data):
    file = xa.Dataset(
        data_vars={"flash_extent": (("longitude", "latitude"), data)},
        coords={"longitude": lon, "latitude": lat},
        attrs={"instrument_ID": "NALMA"}
    )
    return file

def generate_cog(file, variable_name, latitude, longitude):
    file = file[variable_name]
#     file = file[::-1]
    file = file.transpose(latitude, longitude)
    file.rio.set_spatial_dims(x_dim=longitude, y_dim=latitude, inplace=True)
    file.rio.crs
    file.rio.set_crs('epsg:4326')
    name=f"S2A_20160724_135032_27XVB_B{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}.tif"
    cog_path=f"/home/asubedi/test_cog/cogs/{name}"
    print(name)
    file.rio.to_raster(rf'{cog_path}', driver='COG')

In [649]:
def nalma():
    input_file_path = "/home/asubedi/test_cog/NALMA_20230629_235000_600_10src_0.0109deg-dx_flash_extent.nc"
    variable_name = "flash_extent"
    lat='latitude'
    lon='longitude'
    return input_file_path, variable_name, lat, lon

def wtlma():
    input_file_path = "/home/asubedi/test_cog/WTLMA_20170531_235000_600_10src_0.0108deg-dx_flash_extent.nc"
    variable_name = "flash_extent"
    lat='latitude'
    lon='longitude'
    return input_file_path, variable_name, lat, lon

def trmm_lis():
    input_file_path = '/home/asubedi/Desktop/data/raw-files/TRMM-LIS/lis_vhrfc_1998_2013_v01.nc'
    variable_name = "VHRFC_LIS_FRD"
    lat="Latitude"
    lon="Longitude"
    return input_file_path, variable_name, lat, lon

In [653]:
input_file_path, variable_name, latitude, longitude = wtlma()
file = open_file(input_file_path)
lat, lon, data = copy_lat_lon_data(file, latitude, longitude, variable_name)
file = make_new_xarray(lat, lon, data)

In [654]:
file

<xarray.Dataset>
Dimensions:       (longitude: 400, latitude: 400)
Coordinates:
  * longitude     (longitude) float32 -103.6 -103.6 -103.6 ... -99.36 -99.35
  * latitude      (latitude) float32 35.3 35.29 35.28 35.27 ... 31.72 31.71 31.7
Data variables:
    flash_extent  (longitude, latitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    instrument_ID:  NALMA

In [655]:
generate_cog(file, variable_name, latitude, longitude)

S2A_20160724_135032_27XVB_B322.tif


ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_name: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
Warning 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


In [643]:
input_file_path, variable_name, latitude, longitude = trmm_lis()
file = open_file(input_file_path)
# lat, lon, data = copy_lat_lon_data(file, latitude, longitude, variable_name)
# file = make_new_xarray(lat, lon, data)

In [644]:
file

<xarray.Dataset>
Dimensions:        (Longitude: 3600, Latitude: 760)
Coordinates:
  * Longitude      (Longitude) float64 -179.9 -179.8 -179.8 ... 179.9 179.9
  * Latitude       (Latitude) float64 -37.95 -37.85 -37.75 ... 37.75 37.85 37.95
Data variables:
    VHRFC_LIS_FRD  (Latitude, Longitude) float32 ...
    VHRFC_LIS_VT   (Latitude, Longitude) float32 ...
Attributes:
    Title:         LIS Very High Resolution Full Climatology (VHRFC) (1998-2013)
    ATTENTION:     Please read reference below for details.
    Source:        Tropical Rainfall Measuring Mission
    References:    doi...
    Version:       1.0
    Institutions:  University of Sao Paulo (USP), NOAA NESDIS, NASA MSFC
    history:       R. I. Albrecht, Thu Jan 28 13:02:13 2016

In [645]:
file = file[variable_name]
file

<xarray.DataArray 'VHRFC_LIS_FRD' (Latitude: 760, Longitude: 3600)>
[2736000 values with dtype=float32]
Coordinates:
  * Longitude  (Longitude) float64 -179.9 -179.8 -179.8 ... 179.8 179.9 179.9
  * Latitude   (Latitude) float64 -37.95 -37.85 -37.75 ... 37.75 37.85 37.95
Attributes:
    units:      flashes/km^2/year
    long_name:  Mean LIS Flash Rate Density Full Climatology

In [646]:
file = file[::-1]

In [647]:
file

<xarray.DataArray 'VHRFC_LIS_FRD' (Latitude: 760, Longitude: 3600)>
[2736000 values with dtype=float32]
Coordinates:
  * Longitude  (Longitude) float64 -179.9 -179.8 -179.8 ... 179.8 179.9 179.9
  * Latitude   (Latitude) float64 37.95 37.85 37.75 ... -37.75 -37.85 -37.95
Attributes:
    units:      flashes/km^2/year
    long_name:  Mean LIS Flash Rate Density Full Climatology